In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 5,
  "metadata": {
    "kernel_info": {
      "name": "",
      "version": ""
    },
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.10"
    },
    "toc": {
      "nav_menu": {}
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 01_simulation\n",
        "\n",
        "Demonstration notebook: simulate Heston paths, regime-switching, and the hybrid model that multiplies Heston variance by regime-dependent scalars. This notebook:\n",
        "- imports HestonModel, RegimeSwitchingModel, HybridVolModel\n",
        "- simulates 1000 paths for 500 steps\n",
        "- plots sample asset price paths\n",
        "- plots volatility (hybrid) paths and mean volatility\n",
        "- visualizes regime switching sequences (example path and timewise prevalence)\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Basic imports\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "from matplotlib import cm\n",
        "\n",
        "# Import the models from src\n",
        "from src.models.heston import HestonModel\n",
        "from src.models.regime_switching import RegimeSwitchingModel\n",
        "from src.models.hybrid_vol_model import HybridVolModel\n",
        "\n",
        "# Display defaults\n",
        "%matplotlib inline\n",
        "plt.rcParams['figure.figsize'] = (12, 5)\n",
        "plt.rcParams['font.size'] = 12\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1) Configure models and simulation parameters\n",
        "\n",
        "We set up a Heston model for base stochastic volatility, a simple 3-state Markov chain for regimes, and compose them in a HybridVolModel. We keep seeds for reproducibility."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Simulation parameters\n",
        "n_paths = 1000\n",
        "n_steps = 500\n",
        "dt = 1.0 / 252.0  # daily steps approximation\n",
        "seed = 42\n",
        "\n",
        "# Heston parameters (example values)\n",
        "kappa = 1.5      # mean reversion speed of variance\n",
        "theta = 0.04     # long-run variance\n",
        "sigma = 0.3      # vol-of-vol\n",
        "rho = -0.7       # correlation between asset and variance Brownian motions\n",
        "v0 = 0.04        # initial variance\n",
        "s0 = 100.0       # initial asset price\n",
        "r = 0.01         # risk-free rate ~1%\n",
        "\n",
        "heston = HestonModel(kappa=kappa, theta=theta, sigma=sigma, rho=rho, v0=v0, s0=s0, r=r, dt=dt)\n",
        "\n",
        "# Regime switching: 3 regimes (low, medium, high) with multiplicative vol factors\n",
        "P = np.array([\n",
        "    [0.95, 0.04, 0.01],\n",
        "    [0.03, 0.94, 0.03],\n",
        "    [0.02, 0.08, 0.90],\n",
        "])\n",
        "vol_factors = np.array([0.6, 1.0, 1.8])  # multiplicative scaling of variance in each regime\n",
        "regime_model = RegimeSwitchingModel(P=P, vols=vol_factors)\n",
        "\n",
        "hybrid_model = HybridVolModel(heston_model=heston, regime_model=regime_model, vols_are_multiplicative=True)\n",
        "\n",
        "print(f\"Configured HestonModel (v0={v0}, s0={s0}) and RegimeSwitchingModel with {regime_model.n_regimes} regimes\")\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2) Run simulations\n",
        "\n",
        "We call the hybrid simulator which:\n",
        "- simulates Heston S and v paths\n",
        "- simulates per-path regime sequences\n",
        "- applies multiplicative regime scalars to the variance\n",
        "\n",
        "The hybrid simulation returns hybrid variance, asset price paths, and regime indices."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# Run simulation (this may take some seconds depending on numba compilation)\n",
        "hybrid_v_paths, S_paths, regimes_paths = hybrid_model.simulate_hybrid_paths(n_paths=n_paths, n_steps=n_steps, seed=seed)\n",
        "\n",
        "# Sanity checks\n",
        "print(\"S_paths shape:\", S_paths.shape)        # (n_paths, n_steps+1)\n",
        "print(\"hybrid_v_paths shape:\", hybrid_v_paths.shape)  # (n_paths, n_steps+1)\n",
        "print(\"regimes_paths shape:\", regimes_paths.shape)    # (n_paths, n_steps)\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3) Plot sample asset price paths\n",
        "\n",
        "We plot a small subset of Monte Carlo paths for clarity."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "n_plot = 20\n",
        "indices = np.linspace(0, n_paths - 1, n_plot, dtype=int)\n",
        "t = np.arange(0, n_steps + 1) * dt\n",
        "\n",
        "fig, ax = plt.subplots(figsize=(12, 5))\n",
        "for idx in indices:\n",
        "    ax.plot(t, S_paths[idx], lw=0.9, alpha=0.8)\n",
        "ax.set_title(f\"Sample asset price paths (n={n_plot})\")\n",
        "ax.set_xlabel(\"Time (years)\")\n",
        "ax.set_ylabel(\"Asset Price\")\n",
        "ax.grid(alpha=0.3)\n",
        "plt.show()\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4) Plot volatility (variance) paths\n",
        "\n",
        "We visualize the hybrid variance for the same sample of paths and also plot the cross-sectional mean variance across all paths over time."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "fig, axes = plt.subplots(1, 2, figsize=(14, 4))\n",
        "\n",
        "# Left: sample hybrid variance paths\n",
        "for idx in indices:\n",
        "    axes[0].plot(t, hybrid_v_paths[idx], lw=0.9, alpha=0.9)\n",
        "axes[0].set_title(\"Sample hybrid variance paths (v_t)\")\n",
        "axes[0].set_xlabel(\"Time (years)\")\n",
        "axes[0].set_ylabel(\"Variance v_t\")\n",
        "axes[0].grid(alpha=0.3)\n",
        "\n",
        "# Right: mean variance across all paths over time\n",
        "mean_v = hybrid_v_paths.mean(axis=0)\n",
        "axes[1].plot(t, mean_v, color=\"C1\", lw=2)\n",
        "axes[1].fill_between(t, mean_v * 0.8, mean_v * 1.2, color=\"C1\", alpha=0.15)\n",
        "axes[1].set_title(\"Cross-sectional mean hybrid variance over time\")\n",
        "axes[1].set_xlabel(\"Time (years)\")\n",
        "axes[1].set_ylabel(\"Mean variance\")\n",
        "axes[1].grid(alpha=0.3)\n",
        "\n",
        "plt.tight_layout()\n",
        "plt.show()\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5) Visualize regime switching\n",
        "\n",
        "We show (1) the regime time series for the first simulated path and (2) regime prevalence over time across all Monte Carlo paths (stacked area chart)."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "trusted": true
      },
      "source": [
        "# 5.1 Regime sequence for the first path\n",
        "path_idx = 0\n",
        "regimes_first = regimes_paths[path_idx]\n",
        "time_steps = np.arange(0, n_steps) * dt\n",
        "\n",
        "fig, ax = plt.subplots(figsize=(12, 3))\n",
        "ax.step(time_steps, regimes_first, where='post', color='C2')\n",
        "ax.set_ylim(-0.5, regime_model.n_regimes - 0.5)\n",
        "ax.set_yticks(range(regime_model.n_regimes))\n",
        "ax.set_title(f\"Regime sequence for path {path_idx}\")\n",
        "ax.set_xlabel(\"Time (years)\")\n",
        "ax.set_ylabel(\"Regime index\")\n",
        "ax.grid(alpha=0.2)\n",
        "plt.show()\n",
        "\n",
        "# 5.2 Regime prevalence (counts) over time across all paths\n",
        "counts = np.zeros((regime_model.n_regimes, n_steps), dtype=int)\n",
        "for k in range(regime_model.n_regimes):\n",
        "    counts[k, :] = (regimes_paths == k).sum(axis=0)\n",
        "\n",
        "fractions = counts / n_paths\n",
        "fig, ax = plt.subplots(figsize=(12, 4))\n",
        "labels = [f\"regime {i}\" for i in range(regime_model.n_regimes)]\n",
        "colors = cm.get_cmap('tab10')(np.arange(regime_model.n_regimes))\n",
        "ax.stackplot(time_steps, fractions, labels=labels, colors=colors)\n",
        "ax.set_title('Regime prevalence over time (fraction of paths in each regime)')\n",
        "ax.set_xlabel('Time (years)')\n",
        "ax.set_ylabel('Fraction of paths')\n",
        "ax.legend(loc='upper right')\n",
        "ax.grid(alpha=0.15)\n",
        "plt.show()\n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Notes\n",
        "- This notebook demonstrates a simple Monte Carlo experiment. The default parameters are illustrative; calibrate parameters to data for research-grade experiments.\n",
        "- For performance with many paths, ensure numba has compiled the inner loop (first run may include compilation overhead). Consider using smaller n_paths while developing.\n",
        "- The HybridVolModel implementation simulates regimes per path; the visualized prevalence shows how those draws evolve across the Monte Carlo ensemble.\n"
      ]
    }
  ]
}